In [1]:
def read_data(path):
    f = open(path,"r")
    return [[int(i) for i in line.strip()] for line in f.readlines()]

In [2]:
demo_dataset = read_data("demo.txt")
full_dataset = read_data("data.txt")
demo_dataset

[[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
 [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
 [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
 [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
 [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
 [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
 [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
 [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
 [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
 [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]]

# Part 1

In [3]:
from functools import lru_cache

def get_neightbours(matrix, i, j):
    return get_neightbours_cached(i, j, len(matrix), len(matrix[0]))

# @lru_cache(250000)
# caching is not really efficient
def get_neightbours_cached(i: int, j: int, max_i: int, max_j: int):
    neighbours = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
    neighbours = [(x,y) for (x,y) in neighbours if x>=0 and y>=0 and x<max_i and y<max_j]
    return neighbours

In [4]:
def get_value(matrix, i, j):
    return matrix[i][j]

In [5]:
def set_value(matrix, i, j, value):
    matrix[i][j] = value

In [6]:
def insert(todo, p):
    # I do not check for duplicate as it is slow
    todo.append(p)

In [7]:
def update_cost(matrix, current_cost, todo: list):
    (i,j) = todo.pop(0)
    neighbours = get_neightbours(matrix, i, j)
    cost_i_j = get_value(current_cost, i, j)
    for (x,y) in neighbours:
        current = get_value(current_cost, x, y)
        possible = cost_i_j + get_value(matrix, x, y)
        if (possible < current):
            set_value(current_cost, x, y, possible)
            insert(todo, (x, y))

In [8]:
def compute_path(matrix):
    size = max(len(matrix), len(matrix[0]))
    max_path = size * 2 * 10
    current_cost = [[max_path for i in row] for row in matrix]
    current_cost[0][0] = 0 # the starting point
    def get_value(p):
        return current_cost[p[0]][p[1]]
    todo = [(0,0)]
    count = 0
    while not len(todo) == 0:
        count +=1
        # Sorting the todo list make it more efficient (less iterations)
        # but has a big cost so I only do it periodically
        if count % 100 == 0:
            todo.sort(key = get_value)
        update_cost(matrix, current_cost, todo)
    print("steps: " + str(count)) 
    return current_cost[-1][-1]

In [9]:
assert compute_path(demo_dataset) == 40

steps: 136


In [10]:
compute_path(full_dataset)

steps: 12799


503

# Part 2

In [11]:
def reduce(i: int) -> int:
    if i < 10:
        return i
    return reduce(i - 9)

In [12]:
def expand(matrix):
    max_i = len(matrix)
    max_j = len(matrix[0])
    new_matrix = [[0 for j in range(max_j*5)] for i in range(max_i * 5)]
    for m in range(5):
        for n in range(5):
            for i in range(max_i):
                for j in range(max_j):
                    new_matrix[i + m*max_i][j + n*max_j] = reduce(matrix[i][j] + n + m)
    return new_matrix

In [13]:
# check the bottom right number just in case
assert expand(demo_dataset)[-1][-1] == 9

In [14]:
assert compute_path(expand(demo_dataset)) == 315

steps: 3178


In [15]:
big_full = expand(full_dataset)

In [16]:
%%time
compute_path(big_full)

steps: 253337
CPU times: user 906 ms, sys: 0 ns, total: 906 ms
Wall time: 905 ms


2853

In [17]:
# pip install snakeviz

In [18]:
# %load_ext snakeviz

In [19]:
# %%snakeviz
# compute_path(big_full)